<H1> IBM Apply Data Science Capstone - Week 3 Segmenting and Clustering Neighborhoods in Toronto </H1>

<B> Import Libraries </B>

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!conda install -c conda-forge folium
#import folium # map rendering library

print("Library import completed")

Library import completed


In [2]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

print ("folium imported")

folium imported


<B> Load html table form Wikipedia - List of postal codes of Canada

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
result = requests.get(url)
html_file = result.content
 
soup = BeautifulSoup(html_file, "html.parser")

In [5]:
table = soup.find('table', class_='wikitable sortable')

In [6]:
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


<B> Load data into dataframe </B>

In [7]:
df = pd.DataFrame(res, columns=["Postcode","Borough","Neighbourhood"])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [8]:
df.shape

(288, 3)

<B> Merge Neighbourhood with same Postcode and Borough </B>

In [9]:
Postcode = df.Postcode
Borough = df.Borough
Neighbourhood = df.Neighbourhood

unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []



for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)
    
  

num of unique Postal codes: 180


In [10]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.sort_values('Postcode')
df_toronto

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M5P,Central Toronto,"Forest Hill North, Forest Hill West"
2,M4S,Central Toronto,Davisville
3,M5C,Downtown Toronto,St. James Town
4,M4Z,Not assigned,Not assigned
5,M7K,Not assigned,Not assigned
6,M8C,Not assigned,Not assigned
7,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
8,M5Y,Not assigned,Not assigned
9,M3J,North York,"Northwood Park, York University"


In [11]:
df_toronto.shape

(180, 3)

In [12]:
# read csv of 

df_geo = pd.read_csv("https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv") 

df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df_geo.rename(columns = {"Postal Code":"Postcode"}, inplace=True)

df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_geo2=pd.merge(df_toronto, df_geo, on='Postcode', how='inner')

In [15]:
df_geo2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
2,M4S,Central Toronto,Davisville,43.704324,-79.388790
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


In [16]:
#df_geo3 = df_geo2.drop(['Borough_x', 'Borough_y', 'Neighbourhood_x','Neighbourhood_y'], axis =1)
df_geo3=df_geo2

In [17]:
df_geo3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
2,M4S,Central Toronto,Davisville,43.704324,-79.388790
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


In [18]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_geo3['Latitude'], df_geo3['Longitude'], df_geo3['Borough'], df_geo3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [56]:
toronto_data = df_geo3[df_geo3['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()
toronto_data.set_index("Neighbourhood")

(38, 5)


,Postcode,Borough,Latitude,Longitude
Neighbourhood,,,,
"Forest Hill North, Forest Hill West",M5P,Central Toronto,43.696948,-79.411307
Davisville,M4S,Central Toronto,43.704324,-79.388790
St. James Town,M5C,Downtown Toronto,43.651494,-79.375418
"The Annex, North Midtown, Yorkville",M5R,Central Toronto,43.672710,-79.405678
Central Bay Street,M5G,Downtown Toronto,43.657952,-79.387383
"Parkdale, Roncesvalles",M6R,West Toronto,43.648960,-79.456325
"High Park, The Junction South",M6P,West Toronto,43.661608,-79.464763
"The Beaches West, India Bazaar",M4L,East Toronto,43.668999,-79.315572
"Ryerson, Garden District",M5B,Downtown Toronto,43.657162,-79.378937


In [21]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [22]:
CLIENT_ID = 'T1SHYGRI3N4I0XNYW15EXEA0E1E5QOESRY0DJGCNRF2KIL32' # your Foursquare ID
CLIENT_SECRET = 'FUJZSSXA34HOWKVD50VBPNHHY0GH5MPIFE5FJZRACHPYJLXY' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T1SHYGRI3N4I0XNYW15EXEA0E1E5QOESRY0DJGCNRF2KIL32
CLIENT_SECRET:FUJZSSXA34HOWKVD50VBPNHHY0GH5MPIFE5FJZRACHPYJLXY


In [23]:
toronto_data.loc[0, 'Neighbourhood']

'Forest Hill North, Forest Hill West'

In [24]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Forest Hill North, Forest Hill West are 43.6969476, -79.41130720000001.


In [30]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Postcode'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [31]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1710, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,Kay Gardner Beltline Trail,43.700726,-79.410101,Trail
1,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,Forest Hill Road Park,43.697945,-79.406605,Park
2,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,Nikko Sushi Japenese Restaurant,43.700443,-79.407957,Sushi Restaurant
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,Oliver jewelry,43.700374,-79.407644,Jewelry Store
4,M4S,Central Toronto,Davisville,43.704324,-79.388790,Thobors Boulangerie Patisserie Café,43.704514,-79.388616,Café


In [32]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",43,43,43,43,43,43
M4L,East Toronto,"The Beaches West, India Bazaar",22,22,22,22,22,22
M4M,East Toronto,Studio District,37,37,37,37,37,37
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,11,11,11,11,11,11
M4R,Central Toronto,North Toronto West,19,19,19,19,19,19
M4S,Central Toronto,Davisville,37,37,37,37,37,37
M4T,Central Toronto,"Moore Park, Summerhill East",4,4,4,4,4,4


Let's find out how many unique categories can be curated from all the returned venues

In [34]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 240 uniques categories.


In [35]:
venues_df['VenueCategory'].unique()[:50]

array(['Trail', 'Park', 'Sushi Restaurant', 'Jewelry Store', 'Café',
       'Dessert Shop', 'Indian Restaurant', 'Pizza Place',
       'Italian Restaurant', 'Seafood Restaurant', 'Coffee Shop',
       'Sandwich Place', 'Brewery', 'Thai Restaurant', 'Gym',
       'Toy / Game Store', 'Diner', 'Restaurant', 'Gourmet Shop',
       'Greek Restaurant', 'Farmers Market', 'Pharmacy', 'Flower Shop',
       'Burger Joint', 'Asian Restaurant', 'Tennis Court',
       'Fried Chicken Joint', 'Dance Studio', 'Creperie',
       'Japanese Restaurant', 'BBQ Joint', 'Gastropub', 'Food Truck',
       'Cosmetics Shop', 'Middle Eastern Restaurant', 'Church', 'Hotel',
       'Performing Arts Venue', 'Poke Place', 'American Restaurant',
       'Theater', 'Spa', 'Hostel', 'New American Restaurant', 'Beer Bar',
       'Bakery', 'Cocktail Bar', 'Smoothie Shop',
       'Latin American Restaurant', 'Jazz Club'], dtype=object)

<H1>2. Explore Neighborhoods in Toronoto</H1>

In [36]:

# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1710, 243)


,PostalCode,Borough,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M5P,Central Toronto,"Forest Hill North, Forest Hill West",0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,M5P,Central Toronto,"Forest Hill North, Forest Hill West",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5P,Central Toronto,"Forest Hill North, Forest Hill West",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4S,Central Toronto,Davisville,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h1>3. Analyze Each Neighborhood</H1>

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(38, 243)


,PostalCode,Borough,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.023256,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256
2,M4L,East Toronto,"The Beaches West, India Bazaar",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,East Toronto,Studio District,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027
4,M4N,Central Toronto,Lawrence Park,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,Central Toronto,Davisville North,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,Central Toronto,North Toronto West,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632
7,M4S,Central Toronto,Davisville,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.027027,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000


Now let's create the new dataframe and display the top 10 venues for each PostalCode.

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(38, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Coffee Shop,Pub,Neighborhood,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Grocery Store,Bookstore,Brewery,Bubble Tea Shop,Restaurant,Café
2,M4L,East Toronto,"The Beaches West, India Bazaar",Sandwich Place,Pet Store,Brewery,Italian Restaurant,Food & Drink Shop,Steakhouse,Fish & Chips Shop,Light Rail Station,Fast Food Restaurant,Burger Joint
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Brewery,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,M4P,Central Toronto,Davisville North,Pizza Place,Hotel,Clothing Store,Dance Studio,Sandwich Place,Burger Joint,Breakfast Spot,Food & Drink Shop,Asian Restaurant,Park
6,M4R,Central Toronto,North Toronto West,Sporting Goods Shop,Coffee Shop,Yoga Studio,Rental Car Location,Salon / Barbershop,Sandwich Place,Mexican Restaurant,Metro Station,Chinese Restaurant,Fast Food Restaurant
7,M4S,Central Toronto,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Café,Italian Restaurant,Restaurant,Sushi Restaurant,Coffee Shop,Dance Studio,Pharmacy
8,M4T,Central Toronto,"Moore Park, Summerhill East",Restaurant,Gym,Playground,Tennis Court,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Eastern European Restaurant
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,Pub,American Restaurant,Convenience Store,Pizza Place,Medical Center,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant


<H1>Cluster Areas</H1>

Run k-means to cluster the Toronto areas into 5 clusters.

In [40]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 3, 3, 2, 3, 3, 3, 3, 3])

In [62]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_data.copy()

toronto_merged.rename(columns={'Postcode':'PostalCode'}, inplace="True")


# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!


(38, 16)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,0,Park,Jewelry Store,Sushi Restaurant,Trail,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
1,M4S,Central Toronto,Davisville,43.704324,-79.388790,3,Pizza Place,Sandwich Place,Dessert Shop,Café,Italian Restaurant,Restaurant,Sushi Restaurant,Coffee Shop,Dance Studio,Pharmacy
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Restaurant,Café,Hotel,Breakfast Spot,Italian Restaurant,Gastropub,Bakery,Cosmetics Shop,Seafood Restaurant
3,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,3,Coffee Shop,Café,Sandwich Place,Pizza Place,American Restaurant,Cheese Shop,BBQ Joint,Pub,Indian Restaurant,Burger Joint
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Italian Restaurant,Burger Joint,Sandwich Place,Bubble Tea Shop,Café,Bar,Sushi Restaurant,Middle Eastern Restaurant,Spa


In [63]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(38, 16)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,0,Park,Jewelry Store,Sushi Restaurant,Trail,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
22,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Coffee Shop,Pub,Neighborhood,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Italian Restaurant,Burger Joint,Sandwich Place,Bubble Tea Shop,Café,Bar,Sushi Restaurant,Middle Eastern Restaurant,Spa
21,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3,Sporting Goods Shop,Coffee Shop,Yoga Studio,Rental Car Location,Salon / Barbershop,Sandwich Place,Mexican Restaurant,Metro Station,Chinese Restaurant,Fast Food Restaurant
24,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,3,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Pub,Bubble Tea Shop,Burrito Place,Café
25,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,3,Café,Theater,Bakery,Bookstore,Restaurant,Bar,Japanese Restaurant,Nightclub,Noodle House,Comfort Food Restaurant
26,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,3,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Deli / Bodega,Gastropub,Seafood Restaurant,Italian Restaurant,Beer Bar
27,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Restaurant,Gym,Playground,Tennis Court,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Eastern European Restaurant
28,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,3,Coffee Shop,Pub,American Restaurant,Convenience Store,Pizza Place,Medical Center,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant
29,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Grocery Store,Bookstore,Brewery,Bubble Tea Shop,Restaurant,Café


Finally, let's visualize the resulting clusters

In [66]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


<H1> Examine Clusters </H1>

cluster 1

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0,Park,Jewelry Store,Sushi Restaurant,Trail,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Electronics Store


cluster 2

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,East Toronto,1,Health Food Store,Coffee Shop,Pub,Neighborhood,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


cluster 3

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,2,Coffee Shop,Italian Restaurant,Burger Joint,Sandwich Place,Bubble Tea Shop,Café,Bar,Sushi Restaurant,Middle Eastern Restaurant,Spa


cluster 4

In [71]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,3,Sporting Goods Shop,Coffee Shop,Yoga Studio,Rental Car Location,Salon / Barbershop,Sandwich Place,Mexican Restaurant,Metro Station,Chinese Restaurant,Fast Food Restaurant
24,Downtown Toronto,3,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Pub,Bubble Tea Shop,Burrito Place,Café
25,Downtown Toronto,3,Café,Theater,Bakery,Bookstore,Restaurant,Bar,Japanese Restaurant,Nightclub,Noodle House,Comfort Food Restaurant
26,Downtown Toronto,3,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Deli / Bodega,Gastropub,Seafood Restaurant,Italian Restaurant,Beer Bar
27,Central Toronto,3,Restaurant,Gym,Playground,Tennis Court,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Eastern European Restaurant
28,Central Toronto,3,Coffee Shop,Pub,American Restaurant,Convenience Store,Pizza Place,Medical Center,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant
29,East Toronto,3,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Grocery Store,Bookstore,Brewery,Bubble Tea Shop,Restaurant,Café
30,East Toronto,3,Light Rail Station,Farmers Market,Garden Center,Skate Park,Fast Food Restaurant,Spa,Brewery,Burrito Place,Restaurant,Recording Studio
31,Downtown Toronto,3,Coffee Shop,Restaurant,Café,Pizza Place,Italian Restaurant,Pub,Bakery,Pharmacy,Sandwich Place,Deli / Bodega
32,West Toronto,3,Coffee Shop,Café,Pizza Place,Diner,Italian Restaurant,Sushi Restaurant,Latin American Restaurant,Smoothie Shop,Bookstore,Fish & Chips Shop


cluster 5

In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Downtown Toronto,4,Café,Grocery Store,Park,Nightclub,Baby Store,Diner,Athletics & Sports,Restaurant,Italian Restaurant,Convenience Store
10,Downtown Toronto,4,Coffee Shop,Café,Hotel,American Restaurant,Gastropub,Bar,Bakery,Restaurant,Deli / Bodega,Burger Joint


Observations:
Most of the neighborhoods fall into Cluster 4 which are mostly business areas with cafe, restaurants, supermarkets etc. Cluster 1 is just a park, Cluster 2 are healthy food, Cluster 3 coffee shop, and lastly Cluster 5 park and trail.